# BLIP NoCaps Evaluation (Kaggle)

- Bật Internet trong notebook Kaggle, thêm dataset chứa `nocap_val_4500_captions.json` và `nocaps_test_image_info.json` vào tab *Data*.
- Notebook sẽ tự tải ảnh thiếu dựa trên `coco_url`; nếu có lỗi kết nối, xem log và xử lý thủ công.
- GPU Kaggle (P100/V100) chạy ổn với batch size ≤ 4; nếu OOM hãy giảm thêm.

In [ ]:
from pathlib import Path
import json

BASE_DIR = Path("/kaggle/working")
DATASET_ROOT = Path("/kaggle/input/nocaps")
VAL_JSON = DATASET_ROOT / "nocap_val_4500_captions.json"
TEST_JSON = DATASET_ROOT / "nocaps_test_image_info.json"
REPO_URL = "https://github.com/TMG301-Skubidu/Image-Captioning-Vietnamese-BLIP.git"
REPO_DIR = BASE_DIR / "BLIP"
IMAGE_ROOT = BASE_DIR / "nocaps_images"
ANN_ROOT = BASE_DIR / "nocaps_annotations"
OUTPUT_DIR = BASE_DIR / "NoCaps"

for path in [IMAGE_ROOT / "val", IMAGE_ROOT / "test", ANN_ROOT, OUTPUT_DIR]:
    path.mkdir(parents=True, exist_ok=True)

print(f"VAL_JSON exists: {VAL_JSON.exists()} -> {VAL_JSON}")
print(f"TEST_JSON exists: {TEST_JSON.exists()} -> {TEST_JSON}")

In [ ]:
import subprocess
import sys

if not REPO_DIR.exists():
    subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
else:
    print(f"Repository already available at {REPO_DIR}")

commit = subprocess.check_output(["git", "rev-parse", "HEAD"], cwd=REPO_DIR).decode().strip()
print("Repository commit:", commit)
for rel_path in ["eval_nocaps.py", "data/nocaps_dataset.py", "configs/nocaps.yaml"]:
    path = REPO_DIR / rel_path
    status = "OK" if path.exists() else "MISSING"
    print(f"{rel_path}: {status}")

In [ ]:
import subprocess
import sys

requirements_path = REPO_DIR / "requirements.txt"
subprocess.run([sys.executable, "-m", "pip", "install", "-r", str(requirements_path)], check=True)

In [ ]:
import os
from pathlib import Path
import torch
import timm

os.chdir(REPO_DIR)
print("Working directory:", Path.cwd())
print("Torch:", torch.__version__, "CUDA available:", torch.cuda.is_available())
print("Timm:", timm.__version__)

In [ ]:
def summarize_image(entry):
    keys = ["file_name", "id", "domain", "coco_url", "open_images_id"]
    return {k: entry.get(k) for k in keys if k in entry}

with open(VAL_JSON, "r") as f:
    val_raw = json.load(f)
with open(TEST_JSON, "r") as f:
    test_raw = json.load(f)

val_caps = val_raw.get("annotations", [])
print("Val keys:", list(val_raw.keys()))
print("Val images:", len(val_raw["images"]))
print("Val annotations:", len(val_caps))
print("Val sample image:", summarize_image(val_raw["images"][0]))
print("Val sample captions:", [ann["caption"] for ann in val_caps[:3]])

print("Test keys:", list(test_raw.keys()))
print("Test images:", len(test_raw["images"]))
print("Test has annotations:", "annotations" in test_raw)
print("Test sample image:", summarize_image(test_raw["images"][0]))

In [ ]:
import time
import requests
from tqdm import tqdm

session = requests.Session()

def fetch_image(url, dest, retries=3, timeout=20):
    dest.parent.mkdir(parents=True, exist_ok=True)
    last_err = None
    for attempt in range(1, retries + 1):
        try:
            response = session.get(url, timeout=timeout)
            if response.status_code == 200:
                dest.write_bytes(response.content)
                return True, None
            last_err = f"HTTP {response.status_code}"
        except Exception as exc:
            last_err = str(exc)
        time.sleep(min(5, attempt))
    return False, last_err

def convert_split(raw_data, split):
    images = raw_data["images"]
    annotations = raw_data.get("annotations", [])
    id2captions = {}
    for ann in annotations:
        image_id = int(ann["image_id"])
        id2captions.setdefault(image_id, []).append(ann["caption"])
    converted = []
    failures = []
    for img in tqdm(images, desc=f"{split} images", unit="img"):
        file_name = img["file_name"]
        url = img.get("coco_url") or img.get("flickr_url")
        if not url:
            failures.append({"file_name": file_name, "reason": "missing_url"})
            continue
        dest = IMAGE_ROOT / split / file_name
        if not dest.exists():
            success, err = fetch_image(url, dest)
            if not success:
                failures.append({"file_name": file_name, "reason": err})
                continue
        entry = {"image": f"{split}/{file_name}", "img_id": int(img["id"])}
        if id2captions:
            entry["captions"] = id2captions.get(int(img["id"]), [])
        converted.append(entry)
    return converted, failures

In [ ]:
val_entries, val_failures = convert_split(val_raw, "val")
test_entries, test_failures = convert_split(test_raw, "test")

val_out = ANN_ROOT / "nocaps_val.json"
test_out = ANN_ROOT / "nocaps_test.json"

with open(val_out, "w") as f:
    json.dump(val_entries, f)
with open(test_out, "w") as f:
    json.dump(test_entries, f)

failures_log = OUTPUT_DIR / "download_failures.json"
with open(failures_log, "w") as f:
    json.dump({"val": val_failures, "test": test_failures}, f, indent=2)

print(f"Val entries saved: {len(val_entries)} -> {val_out}")
print(f"Val download failures: {len(val_failures)}")
if val_failures[:3]:
    print("Val failure samples:", val_failures[:3])

print(f"Test entries saved: {len(test_entries)} -> {test_out}")
print(f"Test download failures: {len(test_failures)}")
if test_failures[:3]:
    print("Test failure samples:", test_failures[:3])

if val_failures or test_failures:
    print(f"WARNING: some images failed to download. See {failures_log} for details.")
else:
    print("All images downloaded successfully.")

In [ ]:
import ruamel_yaml as yaml

orig_config_path = Path("configs/nocaps.yaml")
with open(orig_config_path, "r") as f:
    config = yaml.load(f, Loader=yaml.Loader)

config["image_root"] = str(IMAGE_ROOT).replace("\\", "/")
config["ann_root"] = str(ANN_ROOT).replace("\\", "/")
config["batch_size"] = min(4, config.get("batch_size", 32))

config_path = Path("/kaggle/working/nocaps_eval.yaml")
with open(config_path, "w") as f:
    yaml.dump(config, f)

print("Config written to:", config_path)
print(json.dumps(config, indent=2))

In [ ]:
!python eval_nocaps.py --config /kaggle/working/nocaps_eval.yaml --output_dir /kaggle/working/NoCaps --device cuda --distributed False

In [ ]:
result_dir = OUTPUT_DIR / "result"
val_result_path = result_dir / "val.json"
test_result_path = result_dir / "test.json"

with open(val_result_path, "r") as f:
    val_result = json.load(f)
with open(test_result_path, "r") as f:
    test_result = json.load(f)

print("Val captions:", len(val_result))
print("Test captions:", len(test_result))
print("Sample val captions:")
for sample in val_result[:5]:
    print(sample)

print("Artifacts saved under:", OUTPUT_DIR)

## Next steps
- Nếu `download_failures.json` không trống, cân nhắc tải thủ công các ảnh đó rồi chạy lại ô chuyển đổi.
- Nén `nocaps_images`, `nocaps_annotations`, `NoCaps/result/test.json` thành dataset Kaggle riêng để notebook lần sau chạy nhanh hơn.
- Upload `NoCaps/result/test.json` lên server NoCaps để nhận điểm chính thức.